In [7]:
%run functions_NoiseStudy.py
%matplotlib 

# global variables
fs = 13
type_plot = ['png', 'svg']

Using matplotlib backend: MacOSX


In [2]:
# actual evaluation time
now = datetime.datetime.now()
today = now.strftime("%Y%m%d")

# output folder
save_dir_plots = 'plots/' + today + '_measurement'
if not os.path.exists(save_dir_plots):
    os.makedirs(save_dir_plots)
    
save_dir_res = 'Results/' + today + '_measurement'
if not os.path.exists(save_dir_res):
    os.makedirs(save_dir_res)

In [26]:
# depth profile microsensor
path_ = '/Volumes/HIS-CAMEL/04measurementData/20201127_Noise-vs-resolution-paper/'
file_ms = path_ + 'Klaus_Optode_noise_study_26-11-2020/O2gradient_experiement/profiles.txt'

# calibration data 
file_calib = 'Results/20210531_calibration/20210531-1455_calibration_1RoI.hdf5'

# measurement data - images of the optode
file_meas = path_ + 'Klaus_Optode_noise_study_26-11-2020/O2gradient_experiement/'

# image cropping to the region of interest (RoI) that should be analyzed
RoI_op = [[(730, 200), (730, 1250), (1290, 1250), (1290, 200)],   # optode1
          [(1560, 200), (1560, 1250), (2100, 1250), (2100, 200)]] # optode2

# Image resolution - determine conversion factor px -> mm 
# user definition using f.ex. imageJ
px2mm, dpi = image_resolution(px=840.6646, dist_mm=30., inch=1.1811)
print('Image resolution: 1cm equals: {:.2f}px, i.e. {:.0f}dpi'.format(px2mm, dpi))

# ................................................................................
# load depth profile(s) of the microsensir for validation of the depth profile
dic_micro = read_microsensor(file_ms=file_ms, encoding='latin-1')

# load measurement data and crop according to the RoI
dint_red, dint_green, dint_ratio = splitImage(path=file_meas, RoI_op=RoI_op)

Image resolution: 1cm equals: 28.02px, i.e. 712dpi


In [740]:
# one example of one optode
inp = 'optode1, set2'
#input('Which optode and which setting shall be visualized? ') # optode1, set2

### Image blur

##### Option-1: Gaussian blur

In [745]:
# preparation lineprofile
surface = (12.9, 12.) # optode 1, optode2
depth_lp = (-4, 4) # depth for 2D line profile 
depth_op =  (-1.5, 1.) # depth for optode excerpt
pos_lp = (7., 3.) # position of the depth profile within the optode
ls_lw = [0] # line with of the drawn depth profile

# kernel size
kernel='gauss'
ls_kernel = [(1,1), (3,3), (5, 5), (7,7), (9,9), (11, 11), (13,13), (15, 15), (17,17),
             (19,19), (21, 21), (25, 25), (31, 31), (35, 35), (41, 41), (45, 45), (51 ,51),
             (55, 55), (61, 61), (65, 65), (71, 71), (75, 75), (81, 81)]  

# .................................................................
# additional information for visualization
arg = dict({'curve lw': 1.5, 'vmin':-5, 'vmax': 130, 'lw': 0.0, 'offset ms': 0.2,
           'aspect': 4, 'cmap': plt.cm.inferno, 'vmin op': 0, 'vmax op': 100,
           'figsize': (12, 5), 'fontsize': 11, 'marker': ['o', 'd', 'x', '.'],
           'colors': ['slategrey', 'darkorange', 'forestgreen', 'k']})

Option-1.1: Gauss filter applied to ratiometric intensity

In [746]:
# determine O2 concentration for the depth profile (1D) for different image blurs
dO2_Rlp = dict(map(lambda k: 
                  (k, O2_lineprofile_compare_v2(inp=inp, surface=surface, kernel=kernel, 
                                                kshape=k, lp=pos_lp, path_calib=file_calib,
                                                dint_ch1=dint_ratio, px2mm=px2mm, 
                                                ls_lw=ls_lw, blur_pos='ratio')), 
                  ls_kernel))

# determine O2 concentration for different image blurs for an excerpt of the optode in 2D
dO2_Roptode = dict(map(lambda k: 
                      (k, O2blur_optode(kshape=k, inp=inp, path_calib=file_calib, 
                                        px2mm=px2mm, kernel=kernel, dint_ch1=dint_ratio, 
                                        surface=surface, depth_min=depth_op[0], 
                                        depth_max=depth_op[1], blur_pos='ratio')),
                      ls_kernel))

In [747]:
# VISUALIZE image blur along line profile
# prepare microsensor for joint visualization
df_ms = prepMS_plot(index_lp=dO2_Rlp[ls_kernel[0]]['vertical'][ls_lw[0]].index,
                    dic_micro=dic_micro, offset=arg['offset ms'])

# plot lineprofile and excerpt of optode 
dimagesR = dict()
for k in ls_kernel:
    fig_lp = plotLP(kshape=k, dO2_lp=dO2_Rlp, dO2_optode=dO2_Roptode, df_ms=df_ms, arg=arg,
                    header_ms=['Depth (mm)', 'Intensity'], depth_lp=depth_lp, 
                    s=inp.split(',')[1].strip(), depth=df_ms['Depth (mm)'])
    dimagesR[k] = fig_lp

/Users/au652733/Python/Project_NoiseResolution/functions_NoiseStudy.py:400: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig_lp = plt.figure(figsize=(arg['figsize']), dpi=100)


In [828]:
# O2 penetration depth
# set O2 level used for calculating the penetration depth in %air
threshold = 5.

# O2 penetration depth
ydepthR = penetration_depth(dO2_lp=dO2_Rlp, ls_kernel=ls_kernel, df_ms=df_ms, 
                           treshold=threshold)

# plotting penetration depth for different kernel sizes and functions
fig_Rpen = plot_penetrationDepth(depth=ydepthR[0], arg=arg,
                                 ls_kernel=[(l[0],round(l[1] / px2mm, 5)) 
                                            for l in ls_kernel])
fig_Rpen.axes[0].set_xlabel('kernel size [mm]')
plt.show()

In [829]:
# SAVE depth profile
now = datetime.datetime.now() 

# .....................................
name_DP_ = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_DepthProfile_ratioIntensity-blurred_' + '-'.join([i.strip() for i in inp.split(',')])
for k in dimagesR.keys():
    name_DP = name_DP_ + '_kernel-' + str(k[0]) + '.'
    for t in type_plot:
        dimagesR[k].savefig(name_DP + t, dpi=300, transparent=False)
        
# .....................................
name_pen = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_PenetrationDepth_ratioIntensity-blurred_' + '-'.join([i.strip() for i in inp.split(',')])
for t in type_plot:
    fig_Rpen.savefig(name_pen + '.' + t, dpi=300, transparent=False)

Option-1.2: Gauss filter applied to individual intensity channels red and green

In [830]:
# determine O2 concentration for the depth profile (1D) for different image blurs
dO2_Slp = dict(map(lambda k: 
                  (k, O2_lineprofile_compare_v2(inp=inp, surface=surface, kernel=kernel, 
                                                kshape=k, lp=pos_lp, path_calib=file_calib,
                                                dint_ch1=dint_red, dint_ch2=dint_green,
                                                px2mm=px2mm, ls_lw=ls_lw, 
                                                blur_pos='single')), ls_kernel))

# determine O2 concentration for different image blurs for an excerpt of the optode in 2D
dO2_Soptode = dict(map(lambda k: 
                      (k, O2blur_optode(kshape=k, inp=inp, path_calib=file_calib, 
                                        px2mm=px2mm, kernel=kernel, dint_ch1=dint_red, 
                                        dint_ch2=dint_green, surface=surface, 
                                        depth_min=depth_op[0], depth_max=depth_op[1], 
                                        blur_pos='single')), ls_kernel))

In [831]:
# VISUALIZE image blur along line profile
# prepare microsensor for joint visualization
df_ms = prepMS_plot(index_lp=dO2_Slp[ls_kernel[0]]['vertical'][ls_lw[0]].index,
                    dic_micro=dic_micro, offset=arg['offset ms'])

# plot lineprofile and excerpt of optode 
dimagesS = dict()
for k in ls_kernel:
    fig_lp = plotLP(kshape=k, dO2_lp=dO2_Slp, dO2_optode=dO2_Soptode, df_ms=df_ms, arg=arg,
                    header_ms=['Depth (mm)', 'Intensity'], depth_lp=depth_lp, 
                    s=inp.split(',')[1].strip(), depth=df_ms['Depth (mm)'])
    dimagesS[k] = fig_lp

/Users/au652733/Python/Project_NoiseResolution/functions_NoiseStudy.py:400: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig_lp = plt.figure(figsize=(arg['figsize']), dpi=100)


In [832]:
# O2 penetration depth
# set O2 level used for calculating the penetration depth in %air
threshold = 5.

# O2 penetration depth
ydepthS = penetration_depth(dO2_lp=dO2_Slp, ls_kernel=ls_kernel, df_ms=df_ms, 
                            treshold=threshold)

# plotting penetration depth for different kernel sizes and functions
fig_Spen = plot_penetrationDepth(depth=ydepthS[0], arg=arg, 
                                 ls_kernel=[(l[0],round(l[1] / px2mm, 5)) 
                                            for l in ls_kernel])
fig_Spen.axes[0].set_xlabel('kernel size [mm]')

plt.show()

In [833]:
# save depth profile
now = datetime.datetime.now() 

# .....................................
name_DP_ = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_DepthProfile_individualChannels-blurred_' + '-'.join([i.strip() for i in inp.split(',')])
for k in dimagesS.keys():
    name_DP = name_DP_ + '_kernel-' + str(k[0]) + '.'
    for t in type_plot:
        dimagesS[k].savefig(name_DP + t, dpi=300, transparent=False)
        
# .....................................
name_pen = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_PenetrationDepth_individualChannels-blurred_' + '-'.join([i.strip() for i in inp.split(',')])
for t in type_plot:
    fig_Spen.savefig(name_pen + '.' + t, dpi=300, transparent=False)

Option-1.3: Gauss filter applied to normalized ratiometric intensity

In [834]:
# determine O2 concentration for the depth profile (1D) for different image blurs
dO2_Nlp = dict(map(lambda k: 
                  (k, O2_lineprofile_compare_v2(inp=inp, surface=surface, kernel=kernel, 
                                                kshape=k, lp=pos_lp, path_calib=file_calib,
                                                dint_ch1=dint_ratio, dint_ch2=None,
                                                px2mm=px2mm, ls_lw=ls_lw, 
                                                blur_pos='norm')), ls_kernel))

In [835]:
# determine O2 concentration for different image blurs for an excerpt of the optode in 2D
dO2_Noptode = dict(map(lambda k: 
                      (k, O2blur_optode(kshape=k, inp=inp, path_calib=file_calib, 
                                        px2mm=px2mm, kernel=kernel, dint_ch1=dint_ratio, 
                                        surface=surface, depth_min=depth_op[0], 
                                        depth_max=depth_op[1], blur_pos='norm')), 
                       ls_kernel))

In [836]:
# VISUALIZE image blur along line profile
# prepare microsensor for joint visualization
df_ms = prepMS_plot(index_lp=dO2_Nlp[ls_kernel[0]]['vertical'][ls_lw[0]].index,
                    dic_micro=dic_micro, offset=arg['offset ms'])

# plot lineprofile and excerpt of optode 
dimagesN = dict()
for k in ls_kernel:
    fig_lp = plotLP(kshape=k, dO2_lp=dO2_Nlp, dO2_optode=dO2_Noptode, df_ms=df_ms, arg=arg,
                    header_ms=['Depth (mm)', 'Intensity'], depth_lp=depth_lp, 
                    s=inp.split(',')[1].strip(), depth=df_ms['Depth (mm)'])
    dimagesN[k] = fig_lp

In [837]:
# O2 penetration depth
# set O2 level used for calculating the penetration depth in %air
threshold = 5.

# O2 penetration depth
ydepthN = penetration_depth(dO2_lp=dO2_Nlp, ls_kernel=ls_kernel, df_ms=df_ms, 
                            treshold=threshold)

# plotting penetration depth for different kernel sizes and functions
fig_Npen = plot_penetrationDepth(depth=ydepthN[0], arg=arg, 
                                 ls_kernel=[(l[0],round(l[1] / px2mm, 5)) 
                                            for l in ls_kernel])
fig_Npen.axes[0].set_xlabel('kernel size [mm]')

plt.show()

In [839]:
# save depth profile
now = datetime.datetime.now() 
name_DP_ = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_DepthProfile_normalizedIntensity-blurred_' + '-'.join([i.strip() for i in inp.split(',')])

# .....................................
for k in dimagesN.keys():
    name_DP = name_DP_ + '_kernel-' + str(k[0]) + '.'
    for t in type_plot:
        dimagesN[k].savefig(name_DP + t, dpi=300, transparent=False)
        
# .....................................
name_pen = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_PenetrationDepth_normalizedIntensity-blurred_' + '-'.join([i.strip() for i in inp.split(',')])
for t in type_plot:
    fig_Npen.savefig(name_pen + '.' + t, dpi=300, transparent=False)

##### Option-2: Savitzky-Golay filter

In [840]:
# preparation lineprofile
surface = (12.9, 12.) # optode 1, optode2
depth_lp = (-4, 4) # depth for 2D line profile 
depth_op =  (-1.5, 1.) # depth for optode excerpt
pos_lp = (7., 3.) # position of the depth profile within the optode
ls_lw = [0] # line with of the drawn depth profile

# kernel size
kernel = 'savgol'

# polyorder, window
ls_kernel = [(2,3), (2, 5), (2,7), (2,9), (2,11), (2,13), (2,15), (2,17), (2,19), (2,21), 
             (2,31), (2,41), (2, 51), (2,61), (2,71), (2,81)]#,
             #(3,5), (3,7), (3,9), (3,11), (3,13), (3,15), (3,17), (3,19), (3,21), 
             #(3,31), (3,41), (3,51), (3,61), (3,71), (3,81)]

# .................................................................
# additional information for visualization
arg = dict({'curve lw': 1.5, 'vmin':-5, 'vmax': 130, 'lw': 0.0, 'offset ms': 0.2,
           'aspect': 4, 'cmap': plt.cm.inferno, 'vmin op': 0, 'vmax op': 100,
           'figsize': (12, 5), 'fontsize': 11, 'marker': ['o', 'd', '.', 'x'],
           'colors': ['slategrey', 'darkorange', 'k', 'forestgreen']})

Option-2.1: Savgol filter applied to ratiometric intensity

In [841]:
# determine O2 concentration for the depth profile (1D) for different image blurs
dO2_Rlp_sav = dict(map(lambda k: 
                  (k, O2_lineprofile_compare_v2(inp=inp, surface=surface, kernel=kernel, 
                                                kshape=k, lp=pos_lp, path_calib=file_calib,
                                                dint_ch1=dint_ratio, dint_ch2=None,
                                                px2mm=px2mm, ls_lw=ls_lw, 
                                                blur_pos='ratio')), ls_kernel))

# determine O2 concentration for different image blurs for an excerpt of the optode in 2D
dO2_Roptode_sav = dict(map(lambda k: 
                      (k, O2blur_optode(kshape=k, inp=inp, path_calib=file_calib, 
                                        px2mm=px2mm, kernel=kernel, dint_ch1=dint_ratio, 
                                        surface=surface, depth_min=depth_op[0], 
                                        depth_max=depth_op[1], blur_pos='ratio')), 
                       ls_kernel))

In [842]:
# VISUALIZE image blur along line profile
# prepare microsensor for joint visualization
df_ms = prepMS_plot(index_lp=dO2_Rlp_sav[ls_kernel[0]]['vertical'][ls_lw[0]].index,
                    dic_micro=dic_micro, offset=arg['offset ms'])

# plot lineprofile and excerpt of optode 
dimagesR_sav = dict()
for k in ls_kernel:
    fig_lp_sav = plotLP(kshape=k, dO2_lp=dO2_Rlp_sav, dO2_optode=dO2_Roptode_sav, arg=arg,
                        df_ms=df_ms, header_ms=['Depth (mm)', 'Intensity'], 
                        depth_lp=depth_lp, s=inp.split(',')[1].strip(), 
                        depth=df_ms['Depth (mm)'])
    dimagesR_sav[k] = fig_lp_sav

In [845]:
# O2 penetration depth
# set O2 level used for calculating the penetration depth in %air
threshold = 5.

# O2 penetration depth
ydepthR_sav = penetration_depth(dO2_lp=dO2_Rlp_sav, ls_kernel=ls_kernel, df_ms=df_ms, 
                                treshold=threshold)

# plotting penetration depth for different kernel sizes and functions
l = [i for i in ydepthR_sav[0].index if 2 in i]
lind = [i for i in ydepthR_sav[0].index if 2 in i]
l_ind = [(l[0],round(l[1] / px2mm, 5)) for l in lind]

fig_Rpen_sav2 = plot_penetrationDepth(depth=ydepthR_sav[0].loc[l], ls_kernel=l_ind, arg=arg)
fig_Rpen_sav2.axes[0].set_xlabel('kernel size [mm]')
fig_Rpen_sav2.text(0.2, 0.92, '2nd degree polynom')
plt.subplots_adjust(top=0.9)

#l3 = [i for i in ydepthR_sav[0].index if 3 in i]
#l3_ind = [i for i in ydepthR_sav[0].index if 3 in i]
#fig_Rpen_sav3 = plot_penetrationDepth(depth=ydepthR_sav[0].loc[l3], ls_kernel=l3_ind, arg=arg)
#fig_Rpen_sav3.text(0.2, 0.92, '3rd degree polynom')
#plt.subplots_adjust(top=0.9)

plt.show()

[(2, 3), (2, 5), (2, 7), (2, 9), (2, 11), (2, 13), (2, 15), (2, 17), (2, 19), (2, 21), (2, 31), (2, 41), (2, 51), (2, 61), (2, 71), (2, 81)]


In [846]:
# save depth profile
now = datetime.datetime.now() 
name_DP_ = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_DepthProfile_ratiometricIntensity-blurred-Savgol_' + '-'.join([i.strip() for i in inp.split(',')])

# .....................................
for k in dimagesR_sav.keys():
    name_DP = name_DP_ + '_poly-' + str(k[0]) + 'nd-window-' +str(k[1])+ '.'
    for t in type_plot:
        dimagesR_sav[k].savefig(name_DP + t, dpi=300, transparent=False)
        
# .....................................
name_pen = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_PenetrationDepth_ratiometricIntensity-blurred-Savgol_' + '-'.join([i.strip() for i in inp.split(',')])
for t in type_plot:
    fig_Rpen_sav2.savefig(name_pen + '-2ndDegree.' + t, dpi=300, transparent=False)
    fig_Rpen_sav3.savefig(name_pen + '-3rdDegree.' + t, dpi=300, transparent=False)

Option-2.2: Savgol filter applied to individual intensity channels red and green

In [847]:
# determine O2 concentration for the depth profile (1D) for different image blurs
dO2_Slp_sav = dict(map(lambda k: 
                  (k, O2_lineprofile_compare_v2(inp=inp, surface=surface, kernel=kernel, 
                                                kshape=k, lp=pos_lp, path_calib=file_calib,
                                                dint_ch1=dint_red, dint_ch2=dint_green, 
                                                px2mm=px2mm, ls_lw=ls_lw, 
                                                blur_pos='single')), ls_kernel))

# determine O2 concentration for different image blurs for an excerpt of the optode in 2D
dO2_Soptode_sav = dict(map(lambda k: 
                      (k, O2blur_optode(kshape=k, inp=inp, path_calib=file_calib, 
                                        px2mm=px2mm, kernel=kernel, dint_ch1=dint_red, 
                                        dint_ch2=dint_green, surface=surface, 
                                        depth_min=depth_op[0], depth_max=depth_op[1], 
                                        blur_pos='single')), ls_kernel))

In [848]:
# VISUALIZE image blur along line profile
# prepare microsensor for joint visualization
df_ms = prepMS_plot(index_lp=dO2_Slp_sav[ls_kernel[0]]['vertical'][ls_lw[0]].index,
                    dic_micro=dic_micro, offset=arg['offset ms'])

# plot lineprofile and excerpt of optode 
dimagesS_sav = dict()
for k in ls_kernel:
    fig_lp_sav = plotLP(kshape=k, dO2_lp=dO2_Slp_sav, dO2_optode=dO2_Soptode_sav, arg=arg,
                        df_ms=df_ms, header_ms=['Depth (mm)', 'Intensity'], 
                        depth_lp=depth_lp, s=inp.split(',')[1].strip(), 
                        depth=df_ms['Depth (mm)'])
    dimagesS_sav[k] = fig_lp_sav

In [849]:
# O2 penetration depth
# set O2 level used for calculating the penetration depth in %air
threshold = 5.

# O2 penetration depth
ydepthS_sav = penetration_depth(dO2_lp=dO2_Slp_sav, ls_kernel=ls_kernel, df_ms=df_ms, 
                                treshold=threshold)

# plotting penetration depth for different kernel sizes and functions
l = [i for i in ydepthS_sav[0].index if 2 in i]
lind = [i for i in ydepthS_sav[0].index if 2 in i]
l_ind = [(l[0],round(l[1] / px2mm, 5)) for l in lind]
fig_Spen_sav2 = plot_penetrationDepth(depth=ydepthS_sav[0].loc[l], ls_kernel=l_ind, arg=arg)
fig_Spen_sav2.axes[0].set_xlabel('kernel size [mm]')
fig_Spen_sav2.text(0.2, 0.92, '2nd degree polynom')
plt.subplots_adjust(top=0.9)

#l3 = [i for i in ydepthS_sav[0].index if 3 in i]
#l3_ind = [i for i in ydepthS_sav[0].index if 3 in i]
#fig_Spen_sav3 = plot_penetrationDepth(depth=ydepthS_sav[0].loc[l3], ls_kernel=l3_ind, arg=arg)
#fig_Spen_sav3.text(0.2, 0.92, '3rd degree polynom')
#plt.subplots_adjust(top=0.9)

plt.show()

In [582]:
# save depth profile
now = datetime.datetime.now() 
name_DP_ = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_DepthProfile_singleChannel-blurred-Savgol_' + '-'.join([i.strip() for i in inp.split(',')])

# .....................................
for k in dimagesS_sav.keys():
    name_DP = name_DP_ + '_poly-' + str(k[0]) + 'nd-window-' +str(k[1])+ '.'
    for t in type_plot:
        dimagesS_sav[k].savefig(name_DP + t, dpi=300, transparent=False)
        
# .....................................
name_pen = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_PenetrationDepth_singleChannel-blurred-Savgol_' + '-'.join([i.strip() for i in inp.split(',')])
for t in type_plot:
    fig_Spen_sav2.savefig(name_pen + '-2ndDegree.' + t, dpi=300, transparent=False)
    fig_Spen_sav3.savefig(name_pen + '-3rdDegree.' + t, dpi=300, transparent=False)

Option-2.3: Savgol filter applied to normalized ratiometric intensity

In [850]:
# determine O2 concentration for the depth profile (1D) for different image blurs
dO2_Nlp_sav = dict(map(lambda k: 
                  (k, O2_lineprofile_compare_v2(inp=inp, surface=surface, kernel=kernel, 
                                                kshape=k, lp=pos_lp, path_calib=file_calib,
                                                dint_ch1=dint_ratio, dint_ch2=None, 
                                                px2mm=px2mm, ls_lw=ls_lw, 
                                                blur_pos='norm')), ls_kernel))

In [851]:
# determine O2 concentration for different image blurs for an excerpt of the optode in 2D
dO2_Noptode_sav = dict(map(lambda k: 
                      (k, O2blur_optode(kshape=k, inp=inp, path_calib=file_calib, 
                                        px2mm=px2mm, kernel=kernel, dint_ch1=dint_ratio, 
                                        dint_ch2=None, surface=surface, 
                                        depth_min=depth_op[0], depth_max=depth_op[1], 
                                        blur_pos='norm')), ls_kernel))

In [852]:
# VISUALIZE image blur along line profile
# prepare microsensor for joint visualization
df_ms = prepMS_plot(index_lp=dO2_Nlp_sav[ls_kernel[0]]['vertical'][ls_lw[0]].index,
                    dic_micro=dic_micro, offset=arg['offset ms'])

# plot lineprofile and excerpt of optode 
dimagesN_sav = dict()
for k in ls_kernel:
    fig_lp_sav = plotLP(kshape=k, dO2_lp=dO2_Nlp_sav, dO2_optode=dO2_Noptode_sav, arg=arg,
                        df_ms=df_ms, header_ms=['Depth (mm)', 'Intensity'], 
                        depth_lp=depth_lp, s=inp.split(',')[1].strip(), 
                        depth=df_ms['Depth (mm)'])
    dimagesN_sav[k] = fig_lp_sav

In [855]:
# O2 penetration depth
# set O2 level used for calculating the penetration depth in %air
threshold = 5.

# O2 penetration depth
ydepthN_sav = penetration_depth(dO2_lp=dO2_Nlp_sav, ls_kernel=ls_kernel, df_ms=df_ms, 
                                treshold=threshold)

# plotting penetration depth for different kernel sizes and functions
l = [i for i in ydepthS_sav[0].index if 2 in i]
lind = [i for i in ydepthS_sav[0].index if 2 in i]
l_ind = [(l[0],round(l[1] / px2mm, 5)) for l in lind]
fig_Npen_sav2 = plot_penetrationDepth(depth=ydepthN_sav[0].loc[l], ls_kernel=l_ind, arg=arg)
fig_Npen_sav2.axes[0].set_xlabel('kernel size [mm]')
fig_Npen_sav2.text(0.2, 0.92, '2nd degree polynom')
plt.subplots_adjust(top=0.9)

#l3 = [i for i in ydepthN_sav[0].index if 3 in i]
#l3_ind = [i for i in ydepthN_sav[0].index if 3 in i]
#fig_Npen_sav3 = plot_penetrationDepth(depth=ydepthN_sav[0].loc[l3], ls_kernel=l3_ind, arg=arg)
#fig_Npen_sav3.text(0.2, 0.92, '3rd degree polynom')
#plt.subplots_adjust(top=0.9)

plt.show()

In [854]:
# save depth profile
now = datetime.datetime.now() 
name_DP_ = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_DepthProfile_normalizedIntensity-blurred-Savgol_' + '-'.join([i.strip() for i in inp.split(',')])

# .....................................
#for k in dimagesS_sav.keys():
#    name_DP = name_DP_ + '_poly-' + str(k[0]) + 'nd-window-' +str(k[1])+ '.'
#    for t in type_plot:
#        dimagesS_sav[k].savefig(name_DP + t, dpi=300, transparent=False)
        
# .....................................
name_pen = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_PenetrationDepth_normalizedIntensity-blurred-Savgol_' + '-'.join([i.strip() for i in inp.split(',')])
for t in type_plot:
    fig_Spen_sav2.savefig(name_pen + '-2ndDegree.' + t, dpi=300, transparent=False)
    #fig_Spen_sav3.savefig(name_pen + '-3rdDegree.' + t, dpi=300, transparent=False)

### Compare penetration depths

In [ ]:
kernel_select = ls_kernel[:10]
fs = 13
nk = len(kernel_select)
d, d1, d2 = 'vertical', 'horizontal', 'square'

In [ ]:
fig, ax = plt.subplots(figsize=(18,3), ncols=3, sharex=True, sharey=True)
ax[0].set_ylabel('O$_2$ penetration depth [mm]', fontsize=fs)
ax[1].set_xlabel('kernel size', fontsize=fs)

ax[0].set_title('single color channel', loc='left', fontsize=fs*0.9)
ax[0].plot(kernel_select, list(ydepthS[0][d].to_numpy())[:nk], marker=arg['marker'][0], 
          fillstyle='none', lw=0.5, ls='-.', color=arg['colors'][0], label=d + ' blur')
ax[0].plot(kernel_select, list(ydepthS[0][d1].to_numpy())[:nk], marker=arg['marker'][0], 
          fillstyle='none', lw=0.5, ls='-.', color=arg['colors'][1], label=d1 + ' blur')
ax[0].plot(kernel_select, list(ydepthS[0][d2].to_numpy())[:nk], marker=arg['marker'][0], 
          fillstyle='none', lw=0.5, ls='-.', color=arg['colors'][2], label=d2 + ' blur')

ax[1].set_title('ratiometric intensity', loc='left', fontsize=fs*0.9)
ax[1].plot(kernel_select, list(ydepthR[0][d].to_numpy())[:nk], marker=arg['marker'][0], 
          fillstyle='none', lw=0.5, ls='-.', color=arg['colors'][0])
ax[1].plot(kernel_select, list(ydepthR[0][d1].to_numpy())[:nk], marker=arg['marker'][0], 
          fillstyle='none', lw=0.5, ls='-.', color=arg['colors'][1])
ax[1].plot(kernel_select, list(ydepthR[0][d2].to_numpy())[:nk], marker=arg['marker'][0], 
          fillstyle='none', lw=0.5, ls='-.', color=arg['colors'][2])

ax[2].set_title('normalized intensity', loc='left', fontsize=fs*0.9)
ax[2].plot(kernel_select, list(ydepthN[0][d].to_numpy())[:nk], marker=arg['marker'][0], 
          fillstyle='none', lw=0.5, ls='-.', color=arg['colors'][0])
ax[2].plot(kernel_select, list(ydepthN[0][d1].to_numpy())[:nk], marker=arg['marker'][0], 
          fillstyle='none', lw=0.5, ls='-.', color=arg['colors'][1])
ax[2].plot(kernel_select, list(ydepthN[0][d2].to_numpy())[:nk], marker=arg['marker'][0], 
          fillstyle='none', lw=0.5, ls='-.', color=arg['colors'][2])

# format legend
handles, labels = ax[0].get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax[0].legend(by_label.values(), by_label.keys(), loc='upper left')


plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(18,3), ncols=3, sharex=True, sharey=True)
ax[0].set_ylabel('O$_2$ penetration depth [mm]', fontsize=fs)
ax[1].set_xlabel('kernel size', fontsize=fs)

ax[0].set_title('vertical blur', loc='left', fontsize=fs*0.9)
ax[0].plot(kernel_select, list(ydepthS[0][d].to_numpy())[:nk], marker=arg['marker'][0], 
          fillstyle='none', lw=0.5, ls='-.', color=arg['colors'][0], label='single channel')
ax[0].plot(kernel_select, list(ydepthR[0][d].to_numpy())[:nk], marker=arg['marker'][1], 
          fillstyle='none', lw=0.5, ls='-.', color=arg['colors'][1], label='ratiometric intensity')
ax[0].plot(kernel_select, list(ydepthN[0][d].to_numpy())[:nk], marker=arg['marker'][2], 
          fillstyle='none', lw=0.5, ls='-.', color=arg['colors'][2], label='norm. intensity')

ax[1].set_title('horizontal blur', loc='left', fontsize=fs*0.9)
ax[1].plot(kernel_select, list(ydepthS[0][d1].to_numpy())[:nk], marker=arg['marker'][0], 
          fillstyle='none', lw=0.5, ls='-.', color=arg['colors'][0])
ax[1].plot(kernel_select, list(ydepthR[0][d1].to_numpy())[:nk], marker=arg['marker'][1], 
          fillstyle='none', lw=0.5, ls='-.', color=arg['colors'][1])
ax[1].plot(kernel_select, list(ydepthN[0][d1].to_numpy())[:nk], marker=arg['marker'][2], 
          fillstyle='none', lw=0.5, ls='-.', color=arg['colors'][2])

ax[2].set_title('square blur', loc='left', fontsize=fs*0.9)
ax[2].plot(kernel_select, list(ydepthS[0][d2].to_numpy())[:nk], marker=arg['marker'][0], 
          fillstyle='none', lw=0.5, ls='-.', color=arg['colors'][0])
ax[2].plot(kernel_select, list(ydepthR[0][d2].to_numpy())[:nk], marker=arg['marker'][1], 
          fillstyle='none', lw=0.5, ls='-.', color=arg['colors'][1])
ax[2].plot(kernel_select, list(ydepthN[0][d2].to_numpy())[:nk], marker=arg['marker'][2], 
          fillstyle='none', lw=0.5, ls='-.', color=arg['colors'][2])

# format legend
handles, labels = ax[0].get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax[0].legend(by_label.values(), by_label.keys(), loc='upper left')

plt.tight_layout()

In [ ]:
# differential image
fig, ax = plt.subplots(figsize=(18, 3), ncols=2, sharex=True, sharey=True) # nrows=2
fig.text(0.017, 0.2, 'O$_2$ penetration depth [mm]', 
         fontsize=fs*0.9, ha='center', rotation='vertical')
#fig.text(0.026, 0.62, 'average level', fontsize=fs*0.9, ha='center', 
#         rotation='vertical')
#fig.text(0.026, 0.17, '', fontsize=fs*0.9, ha='center', 
#         rotation='vertical')
fig.text(0.52, 0.0075, 'kernel size', fontsize=fs, ha='center')

ax[0].set_title('Difference when single color channel was blurred compared to ratiometric intensity', 
                loc='left', fontsize=fs*0.8)
ax[0].plot(kernel_select, list(ydepthS[0][d] - ydepthR[0][d])[:nk], fillstyle='none',
              marker=arg['marker'][0], color=arg['colors'][0], label=d + ' blur')
ax[0].plot(kernel_select, list(ydepthS[0][d1] - ydepthR[0][d1])[:nk], fillstyle='none',
              marker=arg['marker'][1], color=arg['colors'][1], label=d1 + ' blur')
ax[0].plot(kernel_select, list(ydepthS[0][d2] - ydepthR[0][d2])[:nk], fillstyle='none',
              marker=arg['marker'][2], color=arg['colors'][2], label=d2 + ' blur')

ax[1].set_title('Difference when normalized intensity was blurred compared to ratiometric intensity',
                loc='left', fontsize=fs*0.8)
ax[1].plot(kernel_select, list(ydepthN[0][d] - ydepthR[0][d])[:nk], fillstyle='none',
              marker=arg['marker'][0], color=arg['colors'][0])
ax[1].plot(kernel_select, list(ydepthN[0][d1] - ydepthR[0][d1])[:nk], fillstyle='none',
              marker=arg['marker'][1], color=arg['colors'][1])
ax[1].plot(kernel_select, list(ydepthN[0][d2] - ydepthR[0][d2])[:nk], fillstyle='none',
              marker=arg['marker'][2], color=arg['colors'][2])

# difference standard deviation
#ax[0].plot(kernel_select, list(ydepthS[1][d] - ydepthR[1][d])[:nk], fillstyle='none',
#              marker=arg['marker'][0], color=arg['colors'][0], label=d + ' blur')
#ax[0].plot(kernel_select, list(ydepthS[1][d1] - ydepthR[1][d1])[:nk], fillstyle='none',
#              marker=arg['marker'][1], color=arg['colors'][1], label=d1 + ' blur')
#ax[0].plot(kernel_select, list(ydepthS[1][d2] - ydepthR[1][d2])[:nk], fillstyle='none',
#              marker=arg['marker'][2], color=arg['colors'][2], label=d2 + ' blur')

#ax[1].plot(kernel_select, list(ydepthN[1][d] - ydepthR[1][d])[:nk], fillstyle='none',
#              marker=arg['marker'][0], color=arg['colors'][0])
#ax[1].plot(kernel_select, list(ydepthN[1][d1] - ydepthR[1][d1])[:nk], fillstyle='none',
#              marker=arg['marker'][1], color=arg['colors'][1])
#ax[1].plot(kernel_select, list(ydepthN[1][d2] - ydepthR[1][d2])[:nk], fillstyle='none',
#              marker=arg['marker'][2], color=arg['colors'][2])

# format legend
handles, labels = ax[0].get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax[0].legend(by_label.values(), by_label.keys(), loc='lower left')

plt.subplots_adjust(left=0.075, right=0.99, wspace=0.05)
#plt.tight_layout(pad=2)

In [883]:
print('Gauss')
print(ydepthR[0].T[(5,5)], '\n', ydepthR[1].T[(5,5)])
print('')
print('Savgol')
print(ydepthR_sav[0].T[(2,5)], '\n', ydepthR_sav[1].T[(2,5)])

Gauss
vertical       0.124765
horizontal     0.151485
square         0.151893
microsensor    0.113387
Name: (5, 5), dtype: float64 
 vertical       0.085631
horizontal     0.103365
square         0.099686
microsensor    0.113387
Name: (5, 5), dtype: float64

Savgol
vertical       0.125450
horizontal     0.163411
microsensor    0.113387
Name: (2, 5), dtype: float64 
 vertical       0.077220
horizontal     0.096028
microsensor    0.113387
Name: (2, 5), dtype: float64


In [882]:
print('Gauss')
print(ydepthS[0].T[(5,5)], '\n', ydepthS[1].T[(5,5)])
print('')
print('Savgol')
print(ydepthS_sav[0].T[(2,5)], '\n', ydepthS_sav[1].T[(2,5)])

Gauss
vertical       0.124288
horizontal     0.152846
square         0.151866
microsensor    0.113387
Name: (5, 5), dtype: float64 
 vertical       0.085592
horizontal     0.103882
square         0.099675
microsensor    0.113387
Name: (5, 5), dtype: float64

Savgol
vertical       0.124991
horizontal     0.163670
microsensor    0.113387
Name: (2, 5), dtype: float64 
 vertical       0.077326
horizontal     0.096227
microsensor    0.113387
Name: (2, 5), dtype: float64


In [881]:
print('Gauss')
print(ydepthN[0].T[(3,3)], '\n', ydepthN[1].T[(3,3)])
print('')
print('Savgol')
print(ydepthN_sav[0].T[(2,5)], '\n', ydepthN_sav[1].T[(2,5)])

Gauss
vertical       0.124667
horizontal     0.147907
square         0.146839
microsensor    0.113387
Name: (3, 3), dtype: float64 
 vertical       0.082111
horizontal     0.098586
square         0.089801
microsensor    0.113387
Name: (3, 3), dtype: float64

Savgol
vertical       0.125450
horizontal     0.163411
microsensor    0.113387
Name: (2, 5), dtype: float64 
 vertical       0.077220
horizontal     0.096028
microsensor    0.113387
Name: (2, 5), dtype: float64
